In [17]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from time import sleep
import pandas as pd
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


In [18]:
def get_race_table(driver, race_id):
    url = f"https://race.netkeiba.com/race/shutuba.html?race_id={race_id}" # 禁止文字列のため、プロトコルやドメインは略
    driver.get(url)
    sleep(2)
    data = pd.read_html(driver.page_source)
    race_table = data[0]
    return race_table

In [19]:
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
# df = get_race_table(driver, "202405010310") 
df = get_race_table(driver, "202408010411") 


/var/folders/by/b0fpmcl53nz2m_3dxktpq4940000gn/T/ipykernel_64880/2768957501.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(driver.page_source)


In [28]:
df

枠 馬 番   印         馬名  性齢  斤量    騎手     厩舎 馬体重 (増減) 予想 オッズ  人気 お気に入り馬    
     枠 馬 番   印         馬名  性齢  斤量    騎手     厩舎 馬体重 (増減) 予想 オッズ  人気     登録  メモ
0  NaN NaN  --       アルママ  牡8  未定    石川   美浦畠山      NaN   67.2  17    NaN NaN
1  NaN NaN  --    ヴァモスロード  牡5  未定     幸  栗東西園正      NaN   10.2   4    NaN NaN
2  NaN NaN  --   オメガキャプテン  牡4  未定   横山武  美浦大和田      NaN    3.7   2    NaN NaN
3  NaN NaN  --    キタノブライド  牝4  未定   菅原明  美浦高橋裕      NaN    9.2   3    NaN NaN
4  NaN NaN  --    ココリホウオウ  牡6  未定   横山和   美浦古賀      NaN   16.1   5    NaN NaN
5  NaN NaN  --      サヴァイヴ  牡7  未定    田口  栗東清水久      NaN   73.3  19    NaN NaN
6  NaN NaN  --    サトノヘリオス  セ5  未定   モリス   栗東友道      NaN   23.2   8    NaN NaN
7  NaN NaN  --  シャドウフューリー  牡4  未定   鮫島駿  栗東杉山晴      NaN    3.1   1    NaN NaN
8  NaN NaN  --  ステイトダイアデム  牝5  未定  Ｈドイル  美浦和田郎      NaN   20.7   6    NaN NaN
9  NaN NaN  --      トゥーサン  牝5  未定   吉田豊  美浦高橋文      NaN   70.3  18    NaN NaN
10 NaN NaN  --    トーセントラム  牡4  未定     原  美浦小桧山      NaN   52.7  14    NaN NaN
11 NaN NaN  --    ニシノレバンテ  牡5  未定    杉原   美浦栗田      NaN   24.4   9    NaN NaN
12 NaN NaN  --  フリートオブフット  牝5  未定   横山武  美浦奥村武      NaN   29.1  10    NaN NaN
13 NaN NaN  --  プレジャークルーズ  牝4  未定     原   美浦大竹      NaN   37.1  12    NaN NaN
14 NaN NaN  --      ラヴォルタ  牝6  未定    丸山   美浦田村      NaN   21.4   7    NaN NaN
15 NaN NaN  --       ラーテル  セ4  未定    丸田   美浦南田      NaN   56.5  15    NaN NaN
16 NaN NaN  --      ルルローズ  牝6  未定   木幡巧  美浦伊藤大      NaN   67.1  16    NaN NaN
17 NaN NaN  --  レッドアヴァンティ  牡5  未定    大野   美浦尾関      NaN   30.6  11    NaN NaN
18 NaN NaN  --   ロンギングバース  セ6  未定   小林勝   美浦竹内      NaN   49.8  13    NaN NaN

## 三連複の計算

In [13]:
import requests
from bs4 import BeautifulSoup
import time
import csv

def fetch_race_data(race_id):
    url = f"https://db.netkeiba.com/race/{race_id}"
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Error fetching data for race_id {race_id}: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

    soup = BeautifulSoup(response.content.decode("euc-jp", "ignore"), "html.parser")

    race_data = []
    # 馬の情報を含むテーブルの行を取得
    rows = soup.find_all("tr", class_="HorseList")

    for row in rows:
        cols = row.find_all("td")
        if not cols:
            continue

        # 各列から情報を取得
        race_data.append([
            race_id,  # レースID
            cols[3].text.strip(),  # 馬名
            cols[6].text.strip(),  # 騎手
            cols[2].text.strip(),  # 馬番
            cols[7].text.strip(),  # 走破時間
            cols[12].text.strip(),  # オッズ
            cols[10].text.strip(),  # 通過順
            cols[0].text.strip(),  # 着順
            cols[14].text.strip(),  # 体重
            cols[15].text.strip(),  # 体重変化
            cols[4].text.strip(),  # 性別
            cols[5].text.strip(),  # 年齢
            cols[8].text.strip(),  # 斤量
        ])

    return race_data

def save_to_csv(race_data, filename):
    with open(filename, 'w', newline='', encoding="SHIFT-JIS") as file:
        writer = csv.writer(file)
        writer.writerows(race_data)


In [16]:

# 特定のレースIDを指定
race_id = "202408010411"  # この部分を変更して別のレースIDを指定できます

# レースデータを取得
race_data = fetch_race_data(race_id)

# データをCSVに保存（ファイル名は適宜変更してください）
if race_data:
    save_to_csv(race_data, f"data/{race_id}.csv")

In [15]:
race_data

[]